In [1]:
%load_ext autoreload
%autoreload 2

# Import necessary modules 
import numpy as np
import pandas as pd
from itertools import product
import plotly.graph_objs as go
import plotly.express as px
from numpy.linalg import eigvalsh
from plotly.subplots import make_subplots
import torch
import itertools
import concurrent.futures
from tqdm import tqdm

from rbf_volatility_surface import RBFVolatilitySurface
from smoothness_prior import RBFQuadraticSmoothnessPrior
from dataset_sabr import generate_sabr_call_options
from dupire_pinn_trainer import DupirePINNTrainer

In [2]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

# Define the SABR model parameters
alpha = 0.20  # Stochastic volatility parameter
beta = 0.50   # Elasticity parameter
rho = -0.75   # Correlation between asset price and volatility
nu = 1.0      # Volatility of volatility parameter

# Other model parameters
risk_free_rate = np.log(1.02)  # Risk-free interest rate
underlying_price = 1.0         # Current price of the underlying asset

# Generate the dataset using the SABR model and Black-Scholes formula
call_option_dataset = generate_sabr_call_options(
    alpha=alpha,
    beta=beta,
    rho=rho,
    nu=nu,
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    risk_free_rate=risk_free_rate,
    underlying_price=underlying_price
)

# Maturity times and strike prices from the previous product grid setup
hypothetical_maturity_time_list = np.logspace(np.log10(0.01), np.log10(3.1), 100)
hypothetical_strike_price_list = np.logspace(np.log10(0.7), np.log10(1.75), 100)

# Create the product grid of maturity times and strike prices
hypothetical_product_grid = list(product(hypothetical_maturity_time_list, hypothetical_strike_price_list))
hypothetical_maturity_times, hypothetical_strike_prices = zip(*hypothetical_product_grid)
hypothetical_maturity_times, hypothetical_strike_prices = np.array(hypothetical_maturity_times), np.array(hypothetical_strike_prices)

# Reshape the data for 3D surface plotting
hypothetical_maturities_grid = hypothetical_maturity_times.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))  
hypothetical_strikes_grid = hypothetical_strike_prices.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))

In [3]:
n_roots = 350
# n_roots = 10
smoothness_controller = 3.274549162877732e-05

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    smoothness_controller=smoothness_controller,
    random_state=0,
)

# The constant_volatility is set to a reasonable value
constant_volatility = RBFVolatilitySurface.calculate_constant_volatility(
    call_option_dataset["Implied Volatility"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
    risk_free_rate,
    underlying_price
)

sampled_surface_coefficients = smoothness_prior.sample_smooth_surfaces(1000)

In [4]:
# Loop through the sampled coefficients 
sampled_volatilities = []
for coefficients in sampled_surface_coefficients:
    
    # Initialize the RBFVolatilitySurface class for each set of coefficients
    rbf_surface = RBFVolatilitySurface(
        coefficients=coefficients,
        maturity_times=maturity_times,
        strike_prices=strike_prices,
        maturity_std=maturity_std,
        strike_std=strike_std,
        constant_volatility=constant_volatility
    )

    # Generate the volatility surface over the product grid of times and strikes
    surface_volatilities = [
        rbf_surface.implied_volatility_surface(T, K)
        for T, K in product_grid
    ]
    sampled_volatilities.extend(surface_volatilities)

In [5]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [ ]:
batch_size = 1000
pde_loss_coefficient = 1.0
maturity_zero_loss_coefficient = 1.0
strike_zero_loss_coefficient = 0.1
strike_infinity_loss_coefficient = 1.0
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4
pre_train_epochs = 50
fine_tune_epochs = 20
maturity_min = maturity_time_list.min()
maturity_max = maturity_time_list.max()
strike_min = strike_price_list.min()
strike_max = strike_price_list.max()
volatility_mean = np.mean(sampled_volatilities)
volatility_std = np.std(sampled_volatilities)
strike_infinity = 2.5
device = 'cpu'

# Define the hyperparameter grid
hidden_dim_grid = [64, 128, 256]  # Example grid for hidden_dim
n_layers_grid = [2, 4, 8]         # Example grid for n_layers
pre_train_learning_rate_grid = [1e-4, 1e-3, 1e-2]  # Example grid for learning rate

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame()

# Define the grid search
grid = itertools.product(hidden_dim_grid, n_layers_grid, pre_train_learning_rate_grid)

for hidden_dim, n_layers, pre_train_learning_rate in tqdm(grid):
    # Initialize the DupirePINNTrainer class
    trainer = DupirePINNTrainer(
        hidden_dim=hidden_dim,
        n_layers=n_layers,
        batch_size=batch_size,
        pde_loss_coefficient=pde_loss_coefficient,
        maturity_zero_loss_coefficient=maturity_zero_loss_coefficient,
        strike_zero_loss_coefficient=strike_zero_loss_coefficient,
        strike_infinity_loss_coefficient=strike_infinity_loss_coefficient,
        pre_train_learning_rate=pre_train_learning_rate,
        fine_tune_learning_rate=fine_tune_learning_rate,
        pre_train_epochs=pre_train_epochs,
        fine_tune_epochs=fine_tune_epochs,
        maturity_min=maturity_min,
        maturity_max=maturity_max,
        strike_min=strike_min,
        strike_max=strike_max,
        volatility_mean=volatility_mean,
        volatility_std=volatility_std,
        maturity_time_list=maturity_time_list,
        strike_price_list=strike_price_list,
        strike_std=strike_std,
        maturity_std=maturity_std,
        constant_volatility=constant_volatility,
        strike_infinity=strike_infinity,
        device=device
    )

    # Train the model using pre_train
    trainer.pre_train(
        sampled_surface_coefficients=sampled_surface_coefficients,
        experiment_name=f"test_hd_{hidden_dim}_nl_{n_layers}_lr_{pre_train_learning_rate}"
    )

    # Retrieve the last row of the loss history (assuming it's stored in trainer.pre_train_loss_history)
    loss_df = pd.DataFrame(trainer.pre_train_loss_history)
    last_row = loss_df.iloc[-1].copy()

    # Add the configuration as columns in the last row
    last_row['hidden_dim'] = hidden_dim
    last_row['n_layers'] = n_layers
    last_row['pre_train_learning_rate'] = pre_train_learning_rate

    results_df = pd.concat([results_df, pd.DataFrame([last_row])], ignore_index=True)

In [28]:
# Rank the losses for each column (except 'Total Loss')
ranked_losses = results_df.drop(columns=['Total Loss', 'hidden_dim', 'n_layers', 'pre_train_learning_rate']).rank()

ranked_df = results_df.copy()

# Compute the average rank for each configuration
ranked_df['average_rank'] = ranked_losses.mean(axis=1)

# Sort by the average rank (lower is better)
ranked_df = ranked_df.sort_values('average_rank')

# Print the top-ranked configurations
ranked_df

,PDE Loss,Zero Maturity Loss,Zero Strike Loss,Infinity Strike Loss,Total Loss,hidden_dim,n_layers,pre_train_learning_rate,average_rank
7,0.000114,0.000091,0.000040,0.000029,0.000238,64.0,8.0,0.0010,3.25
16,0.000115,0.000109,0.000033,0.000030,0.000258,128.0,8.0,0.0010,4.00
25,0.000096,0.000138,0.000656,0.000035,0.000334,256.0,8.0,0.0010,5.25
4,0.000176,0.000112,0.000260,0.000058,0.000371,64.0,4.0,0.0010,7.50
24,0.000296,0.000110,0.000129,0.000063,0.000482,256.0,8.0,0.0001,8.25
13,0.000174,0.000115,0.000439,0.000075,0.000408,128.0,4.0,0.0010,8.50
15,0.000465,0.000211,0.001115,0.000024,0.000812,128.0,8.0,0.0001,9.75
21,0.000166,0.000435,0.009744,0.000029,0.001604,256.0,4.0,0.0001,10.50
8,0.000192,0.000307,0.000026,0.000169,0.000671,64.0,8.0,0.0100,10.75
1,0.000127,0.000384,0.005440,0.000084,0.001139,64.0,2.0,0.0010,10.75


In [ ]:
hidden_dim = 64
n_layers = 8
batch_size = 1000
pde_loss_coefficient = 1.0
maturity_zero_loss_coefficient = 1.0
strike_zero_loss_coefficient = 1.0
strike_infinity_loss_coefficient = 1.0
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4
pre_train_epochs = 3
fine_tune_epochs = 20
maturity_min = maturity_time_list.min()
maturity_max = maturity_time_list.max()
strike_min = strike_price_list.min()
strike_max = strike_price_list.max()
volatility_mean = np.mean(sampled_volatilities)
volatility_std = np.std(sampled_volatilities)
strike_infinity = 2.5
device = 'cpu'

init_loss = pd.DataFrame()

for i in range(100):
    # Initialize the DupirePINNTrainer class
    trainer = DupirePINNTrainer(
        hidden_dim=hidden_dim,
        n_layers=n_layers,
        batch_size=batch_size,
        pde_loss_coefficient=pde_loss_coefficient,
        maturity_zero_loss_coefficient=maturity_zero_loss_coefficient,
        strike_zero_loss_coefficient=strike_zero_loss_coefficient,
        strike_infinity_loss_coefficient=strike_infinity_loss_coefficient,
        pre_train_learning_rate=pre_train_learning_rate,
        fine_tune_learning_rate=fine_tune_learning_rate,
        pre_train_epochs=pre_train_epochs,
        fine_tune_epochs=fine_tune_epochs,
        maturity_min=maturity_min,
        maturity_max=maturity_max,
        strike_min=strike_min,
        strike_max=strike_max,
        volatility_mean=volatility_mean,
        volatility_std=volatility_std,
        maturity_time_list=maturity_time_list,
        strike_price_list=strike_price_list,
        strike_std=strike_std,
        maturity_std=maturity_std,
        constant_volatility=constant_volatility,
        strike_infinity=strike_infinity,
        device=device
    )

    trainer.pre_train(
        sampled_surface_coefficients=sampled_surface_coefficients,
        experiment_name='test 1'
    )

    init_loss = pd.concat([init_loss, pd.DataFrame(trainer.pre_train_loss_history)], ignore_index=True)

In [55]:
(1 / init_loss).median()

PDE Loss                839.440332
Zero Maturity Loss      914.230596
Zero Strike Loss         32.072821
Infinity Strike Loss    228.623162
Total Loss               25.773987
dtype: float64

In [6]:
hidden_dim = 64
n_layers = 8
batch_size = 1000
pde_loss_coefficient = 850.0
maturity_zero_loss_coefficient = 900.0
strike_zero_loss_coefficient = 30.0
strike_infinity_loss_coefficient = 250.0
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4
pre_train_epochs = 50
fine_tune_epochs = 20
maturity_min = maturity_time_list.min()
maturity_max = maturity_time_list.max()
strike_min = strike_price_list.min()
strike_max = strike_price_list.max()
volatility_mean = np.mean(sampled_volatilities)
volatility_std = np.std(sampled_volatilities)
strike_infinity = 2.5
device = 'cpu'

# Initialize the DupirePINNTrainer class
trainer = DupirePINNTrainer(
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    batch_size=batch_size,
    pde_loss_coefficient=pde_loss_coefficient,
    maturity_zero_loss_coefficient=maturity_zero_loss_coefficient,
    strike_zero_loss_coefficient=strike_zero_loss_coefficient,
    strike_infinity_loss_coefficient=strike_infinity_loss_coefficient,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    maturity_min=maturity_min,
    maturity_max=maturity_max,
    strike_min=strike_min,
    strike_max=strike_max,
    volatility_mean=volatility_mean,
    volatility_std=volatility_std,
    maturity_time_list=maturity_time_list,
    strike_price_list=strike_price_list,
    strike_std=strike_std,
    maturity_std=maturity_std,
    constant_volatility=constant_volatility,
    strike_infinity=strike_infinity,
    device=device
)

trainer.pre_train_with_sampling(
    smoothness_prior=smoothness_prior,
    experiment_name='test 1'
)

Epoch 1/50, Losses: {'PDE Loss': 0.0001953847080380968, 'Zero Maturity Loss': 0.0009452851954847574, 'Zero Strike Loss': 0.07255642116069794, 'Infinity Strike Loss': 0.002272723475471139, 'Total Loss': 3.7617072899564667}
Epoch 2/50, Losses: {'PDE Loss': 0.00409474262265232, 'Zero Maturity Loss': 0.0015272407326847315, 'Zero Strike Loss': 0.06295443326234818, 'Infinity Strike Loss': 0.0053780339658260345, 'Total Loss': 8.088189377066362}
Epoch 3/50, Losses: {'PDE Loss': 8.323679418755158e-05, 'Zero Maturity Loss': 0.0012462212471291423, 'Zero Strike Loss': 0.06133729964494705, 'Infinity Strike Loss': 0.001744386157952249, 'Total Loss': 3.468565906521516}
Epoch 4/50, Losses: {'PDE Loss': 0.0005834500663268324, 'Zero Maturity Loss': 0.0004763662873301655, 'Zero Strike Loss': 0.053102560341358185, 'Infinity Strike Loss': 0.00027722667437046766, 'Total Loss': 2.587045706497589}
Epoch 5/50, Losses: {'PDE Loss': 0.0005063733746139666, 'Zero Maturity Loss': 0.0003294216876383871, 'Zero Strike

In [7]:
loss_history = pd.DataFrame(trainer.pre_train_loss_history)

# Create a subplot figure with 2x2 grid for individual losses, and a third row spanning the entire width for total loss
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=("PDE Loss", "Zero Maturity Loss", "Zero Strike Loss", "Infinity Strike Loss", "Total Loss"),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'scatter'}],
           [{'colspan': 2, 'type': 'scatter'}, None]],
    vertical_spacing=0.1,
    horizontal_spacing=0.1
)

# Add traces for individual losses
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["PDE Loss"], mode="lines", name="PDE Loss"), row=1, col=1)
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Zero Maturity Loss"], mode="lines", name="Zero Maturity Loss"), row=1, col=2)
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Zero Strike Loss"], mode="lines", name="Zero Strike Loss"), row=2, col=1)
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Infinity Strike Loss"], mode="lines", name="Infinity Strike Loss"), row=2, col=2)

# Add a trace for the total loss spanning the entire third row
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Total Loss"], mode="lines", name="Total Loss"), row=3, col=1)

# Update the layout to include 'Iterations' as the x-axis name for each subplot
fig.update_xaxes(title_text="Iterations", row=1, col=1)
fig.update_xaxes(title_text="Iterations", row=1, col=2)
fig.update_xaxes(title_text="Iterations", row=2, col=1)
fig.update_xaxes(title_text="Iterations", row=2, col=2)
fig.update_xaxes(title_text="Iterations", row=3, col=1)  # The third row spans two columns

# Update the layout
fig.update_layout(height=900, width=900, title_text="PINN Training Losses", showlegend=False)

# Show the plot
fig.show()